<a href="https://colab.research.google.com/github/sadiashoaib415/Quantum_Computer/blob/main/BB84_Protocol_with_QEBR01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer --quiet

Given Code

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# -------------------------
# Parameters
# -------------------------
n = 40
QBER_THRESHOLD = 0.11  # 11%

# Alice
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.randint(2, size=n)

# Eve (Interference)
eve_bases = np.random.randint(2, size=n)

# Bob
bob_bases = np.random.randint(2, size=n)

circuits = []
transpiled = []

for i in range(n):
    qc = QuantumCircuit(1, 1)

    # ----- Alice prepares -----
    if alice_bits[i] == 1:
        qc.x(0)
    if alice_bases[i] == 1:
        qc.h(0)

    # ----- Eve intercepts -----
    if eve_bases[i] == 1:
        qc.h(0)
    qc.measure(0, 0)
    qc.reset(0)

    # Eve resends
    if alice_bits[i] == 1:
        qc.x(0)
    if eve_bases[i] == 1:
        qc.h(0)

    # ----- Bob measures -----
    if bob_bases[i] == 1:
        qc.h(0)
    qc.measure(0, 0)

    circuits.append(qc)

# Setup simulator
backend = AerSimulator()

# Transpile circuits for the simulator
for qc in circuits:
    transpiled.append(transpile(qc, backend))

# Run all circuits
job = backend.run(transpiled, shots=1)
result = job.result()

# Bob results
bob_results = []
for qc in transpiled:
    counts = result.get_counts(qc)
    bob_results.append(int(list(counts.keys())[0]))

# Key sifting
alice_key = []
bob_key = []

for i in range(n):
    if alice_bases[i] == bob_bases[i]:
        alice_key.append(alice_bits[i])
        bob_key.append(bob_results[i])

# -------------------------
# QBER Calculation
# -------------------------
errors = sum(a != b for a, b in zip(alice_key, bob_key))
qber = errors / len(alice_key) if alice_key else 0

# -------------------------
# Decision
# -------------------------
print("Final key length:", len(alice_key))
print("Errors detected:", errors)
print("QBER:", round(qber, 3))

if qber > QBER_THRESHOLD:
    print("⚠️ QBER too high — Eavesdropping detected!")
    print("❌ Key exchange ABORTED")
else:
    print("✅ QBER acceptable")
    print("🔐 Secure key established")
    print("Shared key:", alice_key)

Final key length: 20
Errors detected: 6
QBER: 0.3
⚠️ QBER too high — Eavesdropping detected!
❌ Key exchange ABORTED


The code is incorrect because Eve re-prepares the qubit using Alice’s original bit instead of her own measurement result, which unrealistically gives Eve perfect knowledge.
This violates the intercept–resend principle of BB84 and suppresses the quantum disturbance that should produce detectable QBER.

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# -------------------------
# Parameters
# -------------------------
n = 40
QBER_THRESHOLD = 0.11  # 11%

# Alice
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.randint(2, size=n)

# Eve (Interference)
eve_bases = np.random.randint(2, size=n)

# Bob
bob_bases = np.random.randint(2, size=n)

backend = AerSimulator()

bob_results = []
eve_results = []

# =========================
# ALICE → EVE
# =========================
for i in range(n):
    qc = QuantumCircuit(1, 1)

    # Alice prepares
    if alice_bits[i] == 1:
        qc.x(0)
    if alice_bases[i] == 1:
        qc.h(0)

    # Eve measures
    if eve_bases[i] == 1:
        qc.h(0)
    qc.measure(0, 0)

    transpiled = transpile(qc, backend)
    job = backend.run(transpiled, shots=1)
    result = job.result()
    counts = result.get_counts()
    eve_results.append(int(list(counts.keys())[0]))

# =========================
# EVE → BOB
# =========================
for i in range(n):
    qc = QuantumCircuit(1, 1)

    # Eve resends her measured bit
    if eve_results[i] == 1:
        qc.x(0)
    if eve_bases[i] == 1:
        qc.h(0)

    # Bob measures
    if bob_bases[i] == 1:
        qc.h(0)
    qc.measure(0, 0)

    transpiled = transpile(qc, backend)
    job = backend.run(transpiled, shots=1)
    result = job.result()
    counts = result.get_counts()
    bob_results.append(int(list(counts.keys())[0]))

# =========================
# KEY SIFTING
# =========================
alice_key = []
bob_key = []

for i in range(n):
    if alice_bases[i] == bob_bases[i]:
        alice_key.append(alice_bits[i])
        bob_key.append(bob_results[i])

# =========================
# QBER
# =========================
errors = sum(a != b for a, b in zip(alice_key, bob_key))
qber = errors / len(alice_key) if alice_key else 0

# =========================
# DECISION
# =========================
print("Final key length:", len(alice_key))
print("Errors detected:", errors)
print("QBER:", round(qber, 3))

if qber > QBER_THRESHOLD:
    print("⚠️ QBER too high — Eavesdropping detected!")
    print("❌ Key exchange ABORTED")
else:
    print("✅ QBER acceptable")
    print("🔐 Secure key established")
    print("Shared key:", alice_key)


Final key length: 22
Errors detected: 8
QBER: 0.364
⚠️ QBER too high — Eavesdropping detected!
❌ Key exchange ABORTED


Code runs for 10 times

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# -------------------------
# Parameters
# -------------------------
n = 40
QBER_THRESHOLD = 0.11
RUNS = 10

backend = AerSimulator()

# ===== Summary trackers =====
abort_count = 0
accept_count = 0
qber_list = []

for run in range(1, RUNS + 1):

    # Alice
    alice_bits = np.random.randint(2, size=n)
    alice_bases = np.random.randint(2, size=n)

    # Eve
    eve_bases = np.random.randint(2, size=n)

    # Bob
    bob_bases = np.random.randint(2, size=n)

    bob_results = []
    eve_results = []

    # =========================
    # ALICE → EVE
    # =========================
    for i in range(n):
        qc = QuantumCircuit(1, 1)

        if alice_bits[i] == 1:
            qc.x(0)
        if alice_bases[i] == 1:
            qc.h(0)

        if eve_bases[i] == 1:
            qc.h(0)
        qc.measure(0, 0)

        transpiled = transpile(qc, backend)
        job = backend.run(transpiled, shots=1)
        result = job.result()
        counts = result.get_counts()
        eve_results.append(int(list(counts.keys())[0]))

    # =========================
    # EVE → BOB
    # =========================
    for i in range(n):
        qc = QuantumCircuit(1, 1)

        if eve_results[i] == 1:
            qc.x(0)
        if eve_bases[i] == 1:
            qc.h(0)

        if bob_bases[i] == 1:
            qc.h(0)
        qc.measure(0, 0)

        transpiled = transpile(qc, backend)
        job = backend.run(transpiled, shots=1)
        result = job.result()
        counts = result.get_counts()
        bob_results.append(int(list(counts.keys())[0]))

    # =========================
    # KEY SIFTING
    # =========================
    alice_key = []
    bob_key = []

    for i in range(n):
        if alice_bases[i] == bob_bases[i]:
            alice_key.append(alice_bits[i])
            bob_key.append(bob_results[i])

    # =========================
    # QBER
    # =========================
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    qber = errors / len(alice_key) if alice_key else 0
    qber_list.append(qber)

    # =========================
    # OUTPUT (PER RUN)
    # =========================
    print(f"Run {run}: "
          f"Key length = {len(alice_key)}, "
          f"Errors = {errors}, "
          f"QBER = {round(qber, 3)}")

    if qber > QBER_THRESHOLD:
        print("  ⚠️ Eavesdropping detected — ABORTED\n")
        abort_count += 1
    else:
        print("  ✅ Key accepted\n")
        accept_count += 1

# =========================
# FINAL SUMMARY
# =========================
print("========== SUMMARY ==========")
print("Total runs:", RUNS)
print("Accepted keys:", accept_count)
print("Aborted keys:", abort_count)
print("Average QBER:", round(sum(qber_list) / RUNS, 3))
print("Max QBER:", round(max(qber_list), 3))
print("Min QBER:", round(min(qber_list), 3))
print("=============================")


Run 1: Key length = 22, Errors = 4, QBER = 0.182
  ⚠️ Eavesdropping detected — ABORTED

Run 2: Key length = 21, Errors = 5, QBER = 0.238
  ⚠️ Eavesdropping detected — ABORTED

Run 3: Key length = 20, Errors = 7, QBER = 0.35
  ⚠️ Eavesdropping detected — ABORTED

Run 4: Key length = 22, Errors = 8, QBER = 0.364
  ⚠️ Eavesdropping detected — ABORTED

Run 5: Key length = 17, Errors = 2, QBER = 0.118
  ⚠️ Eavesdropping detected — ABORTED

Run 6: Key length = 19, Errors = 5, QBER = 0.263
  ⚠️ Eavesdropping detected — ABORTED

Run 7: Key length = 26, Errors = 12, QBER = 0.462
  ⚠️ Eavesdropping detected — ABORTED

Run 8: Key length = 15, Errors = 2, QBER = 0.133
  ⚠️ Eavesdropping detected — ABORTED

Run 9: Key length = 19, Errors = 4, QBER = 0.211
  ⚠️ Eavesdropping detected — ABORTED

Run 10: Key length = 21, Errors = 9, QBER = 0.429
  ⚠️ Eavesdropping detected — ABORTED

========== SUMMARY ==========
Total runs: 10
Accepted keys: 0
Aborted keys: 10
Average QBER: 0.275
Max QBER: 0.462
Min 

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# -------------------------
# Parameters
# -------------------------
n = 40
QBER_THRESHOLD = 0.11
RUNS = 10

backend = AerSimulator()

# ===== Summary trackers =====
abort_count = 0
accept_count = 0
qber_list = []

for run in range(1, RUNS + 1):

    # Alice
    alice_bits = np.random.randint(2, size=n)
    alice_bases = np.random.randint(2, size=n)

    # Eve
    eve_bases = np.random.randint(2, size=n)

    # Bob
    bob_bases = np.random.randint(2, size=n)

    bob_results = []
    eve_results = []

    # =========================
    # ALICE → EVE
    # =========================
    for i in range(n):
        qc = QuantumCircuit(1, 1)

        if alice_bits[i] == 1:
            qc.x(0)
        if alice_bases[i] == 1:
            qc.h(0)

        if eve_bases[i] == 1:
            qc.h(0)
        qc.measure(0, 0)

        transpiled = transpile(qc, backend)
        job = backend.run(transpiled, shots=1)
        result = job.result()
        counts = result.get_counts()
        eve_results.append(int(list(counts.keys())[0]))

    # =========================
    # EVE → BOB
    # =========================
    for i in range(n):
        qc = QuantumCircuit(1, 1)

        if eve_results[i] == 1:
            qc.x(0)
        if eve_bases[i] == 1:
            qc.h(0)

        if bob_bases[i] == 1:
            qc.h(0)
        qc.measure(0, 0)

        transpiled = transpile(qc, backend)
        job = backend.run(transpiled, shots=1)
        result = job.result()
        counts = result.get_counts()
        bob_results.append(int(list(counts.keys())[0]))

    # =========================
    # KEY SIFTING
    # =========================
    alice_key = []
    bob_key = []

    for i in range(n):
        if alice_bases[i] == bob_bases[i]:
            alice_key.append(alice_bits[i])
            bob_key.append(bob_results[i])

    # =========================
    # QBER
    # =========================
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    qber = errors / len(alice_key) if alice_key else 0
    qber_list.append(qber)

    # =========================
    # OUTPUT (PER RUN)
    # =========================
    print(f"Run {run}: "
          f"Key length = {len(alice_key)}, "
          f"Errors = {errors}, "
          f"QBER = {round(qber, 3)}")

    if qber > QBER_THRESHOLD:
        print("  ⚠️ Eavesdropping detected — ABORTED\n")
        abort_count += 1
    else:
        print("  ✅ Key accepted\n")
        accept_count += 1

# =========================
# FINAL SUMMARY
# =========================
print("========== SUMMARY ==========")
print("Total runs:", RUNS)
print("Accepted keys:", accept_count)
print("Aborted keys:", abort_count)
print("Average QBER:", round(sum(qber_list) / RUNS, 3))
print("Max QBER:", round(max(qber_list), 3))
print("Min QBER:", round(min(qber_list), 3))
print("=============================")


Run 1: Key length = 23, Errors = 7, QBER = 0.304
  ⚠️ Eavesdropping detected — ABORTED

Run 2: Key length = 21, Errors = 9, QBER = 0.429
  ⚠️ Eavesdropping detected — ABORTED

Run 3: Key length = 19, Errors = 4, QBER = 0.211
  ⚠️ Eavesdropping detected — ABORTED

Run 4: Key length = 13, Errors = 3, QBER = 0.231
  ⚠️ Eavesdropping detected — ABORTED

Run 5: Key length = 18, Errors = 5, QBER = 0.278
  ⚠️ Eavesdropping detected — ABORTED

Run 6: Key length = 20, Errors = 2, QBER = 0.1
  ✅ Key accepted

Run 7: Key length = 19, Errors = 8, QBER = 0.421
  ⚠️ Eavesdropping detected — ABORTED

Run 8: Key length = 22, Errors = 4, QBER = 0.182
  ⚠️ Eavesdropping detected — ABORTED

Run 9: Key length = 24, Errors = 8, QBER = 0.333
  ⚠️ Eavesdropping detected — ABORTED

Run 10: Key length = 25, Errors = 8, QBER = 0.32
  ⚠️ Eavesdropping detected — ABORTED

========== SUMMARY ==========
Total runs: 10
Accepted keys: 1
Aborted keys: 9
Average QBER: 0.281
Max QBER: 0.429
Min QBER: 0.1
